# 使用 Langchain 和 GLM 完成简单的简历信息抽取任务。

**This tutorial is Only in Chinese explanation**

本代码，我将使用 Langchain 配合 GLM-4 完成 Word 文档的简历信息抽取任务。


## 1. 配置相关环境
由于需要安装部分依赖，这里我们需要安装必要的依赖，如果你已经安装了这些依赖，你可以跳过这个步骤。

In [1]:
!pip install langchain unstructured python-docx  

     ---------------------------------------- 0.0/83.2 kB ? eta -:--:--
     ---- ----------------------------------- 10.2/83.2 kB ? eta -:--:--
     ---- ----------------------------------- 10.2/83.2 kB ? eta -:--:--
     -------------- ----------------------- 30.7/83.2 kB 262.6 kB/s eta 0:00:01
     -------------- ----------------------- 30.7/83.2 kB 262.6 kB/s eta 0:00:01
     -------------- ----------------------- 30.7/83.2 kB 262.6 kB/s eta 0:00:01
     -------------- ----------------------- 30.7/83.2 kB 262.6 kB/s eta 0:00:01
     -------------- ----------------------- 30.7/83.2 kB 262.6 kB/s eta 0:00:01
     -------------- ----------------------- 30.7/83.2 kB 262.6 kB/s eta 0:00:01
     -------------- ----------------------- 30.7/83.2 kB 262.6 kB/s eta 0:00:01
     -------------- ----------------------- 30.7/83.2 kB 262.6 kB/s eta 0:00:01
     -------------- ----------------------- 30.7/83.2 kB 262.6 kB/s eta 0:00:01
     -------------- ----------------------- 30.7/83.2 kB 262.6

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
types-requests 2.31.0.20240311 requires urllib3>=2, but you have urllib3 1.26.18 which is incompatible.


   ------------------------------------ --- 1.7/1.8 MB 17.6 kB/s eta 0:00:09
   ------------------------------------ --- 1.7/1.8 MB 17.6 kB/s eta 0:00:09
   ------------------------------------ --- 1.7/1.8 MB 17.6 kB/s eta 0:00:09
   ------------------------------------ --- 1.7/1.8 MB 17.6 kB/s eta 0:00:09
   ------------------------------------ --- 1.7/1.8 MB 17.6 kB/s eta 0:00:09
   ------------------------------------ --- 1.7/1.8 MB 17.6 kB/s eta 0:00:09
   ------------------------------------ --- 1.7/1.8 MB 17.6 kB/s eta 0:00:09
   ------------------------------------ --- 1.7/1.8 MB 17.6 kB/s eta 0:00:09
   ------------------------------------ --- 1.7/1.8 MB 17.6 kB/s eta 0:00:09
   ------------------------------------ --- 1.7/1.8 MB 17.6 kB/s eta 0:00:09
   ------------------------------------ --- 1.7/1.8 MB 17.6 kB/s eta 0:00:09
   ------------------------------------ --- 1.7/1.8 MB 17.6 kB/s eta 0:00:09
   ------------------------------------ --- 1.7/1.8 MB 17.6 kB/s eta 0:00:09

接着，我们需要将我们的 API_KEY 配置到环境变量中，用于调用 GLM-4 模型。

In [4]:
import os

#os.environ["ZHIPUAI_API_KEY"] = "your api key"

## 2. 读取简历文档

我们需要将简历的文档用 Langchain 的 UnstructuredWordDocumentLoader 读入，并填充到我们的模板中。

In [5]:
from langchain_community.document_loaders import UnstructuredWordDocumentLoader

loader = UnstructuredWordDocumentLoader("data/resume.docx")
data = loader.load()

[nltk_data] Error loading punkt: <urlopen error [Errno 11004]
[nltk_data]     getaddrinfo failed>


LookupError: 
**********************************************************************
  Resource [93mpunkt[0m not found.
  Please use the NLTK Downloader to obtain the resource:

  [31m>>> import nltk
  >>> nltk.download('punkt')
  [0m
  For more information see: https://www.nltk.org/data.html

  Attempted to load [93mtokenizers/punkt/english.pickle[0m

  Searched in:
    - 'C:\\Users\\jay/nltk_data'
    - 'd:\\sdk\\Anaconda3\\envs\\zhipu\\nltk_data'
    - 'd:\\sdk\\Anaconda3\\envs\\zhipu\\share\\nltk_data'
    - 'd:\\sdk\\Anaconda3\\envs\\zhipu\\lib\\nltk_data'
    - 'C:\\Users\\jay\\AppData\\Roaming\\nltk_data'
    - 'C:\\nltk_data'
    - 'D:\\nltk_data'
    - 'E:\\nltk_data'
    - ''
**********************************************************************


设定好模板，这个模板将会作为我们的系统提示词。我们将使用 Langchain 的 ChatPromptTemplate 来构建这个模板。其中，{resume} 将会被我们的简历内容填充。

In [9]:
from langchain_core.prompts import SystemMessagePromptTemplate
from langchain.schema import HumanMessage, AIMessage
from langchain.prompts import ChatPromptTemplate

system_prompt = """
你是 ZhipuAI 的 人事资源管理部门的优秀员工，现在我需要你帮我阅读简历并筛选出合适的人才，请你基于我提供的简历，对简历进行细节的分析，抓取相关的资料并回答我提出的问题。
现在，我将会将简历以文字的形式给你提供，具体内容如下:

<resume>
{resume}
</resume>

请你根据我的简历，开始回答我的问题吧。请注意我的提问的内容和我需要你回答的格式，我们开始吧：
"""

question_prompt = [
    "候选人读过哪些大学？",
    # "请帮我提取候选人简历中的关键信息，用JSON格式返回给我，我需要的字段是：姓名、性别、年龄、学历、工作年限、工作经历、项目经历、技能、个人优势、个人缺点、兴趣爱好；简历中没有提到的字段也要输出，但字段值为空。json的key可以使用中文，value的长度不要超过100个字符，如果字段值太长，请对内容进行总结摘要再输出。例如工作经历可以只保留公司名称和职位，工作经历和项目经历可以只保留项目名称和项目描述",
    # "你怎么评价这个候选人，从他现有的资历、技术能力、工作态度、发展潜力进行分析。我们公司目前想招聘一个3-5年工作经验有一定的发展潜力的员工，请结合对候选人的分析和我的招聘需求判断我是否应该给他面试机会？"
]

chat_template = ChatPromptTemplate.from_messages(
    [
        SystemMessagePromptTemplate.from_template(system_prompt),
    ]
)
messages = chat_template.format_messages(resume=data)

接着，我们就可以调用 GLM-4 模型，通过模型对简历进行抓取和提取关键信息，获得有效的内容和答案。

In [10]:
from langchain_community.chat_models import ChatZhipuAI

for question in question_prompt:
    messages = chat_template.format_messages(resume=data)
    messages.append(
        HumanMessage(
            content=question
        )
    )
    llm = ChatZhipuAI(
        temperature=0.01,
        model="glm-4",
        max_tokens=8192,
        stream=False,
    )
    messages.append(
        AIMessage(
            content=llm(messages).content
        )
    )
    print(messages[-1].content)

候选人李明浩读过的大学有：

1. 清华大学，专业是计算机科学与技术。
2. 北京邮电大学，专业是信息技术。


## 3. 结果分析

通过大模型，我们可以顺利的抽取出简历中的关键信息，包括教育背景、工作经历等。这样，我们就可以通过简单的代码，完成简历信息的抽取任务。
这是一个开放性的demo，意味着你可以自己选择其他任务来接着完成这个场景的研究。